In [0]:
# Modules that contains our Linear Regression & Random Forest Algorithms
from pyspark.ml.regression import LinearRegression , RandomForestRegressor
# Module with the Evaluator
from pyspark.ml.evaluation import RegressionEvaluator 
# One Hot Encode Module
from pyspark.ml.feature import VectorAssembler

In [0]:
# Import the others modules
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [0]:
# df1 = spark.read.csv("dbfs:/FileStore/YOUR_DATASET_directory/Carros.csv" ,inferSchema= True ,header = True , sep = ";")

In [0]:
# df1.printSchema()
# df1.show()

root
-- Consumo: integer (nullable = true)
-- Cilindros: integer (nullable = true)
-- Cilindradas: integer (nullable = true)
-- RelEixoTraseiro: integer (nullable = true)
-- Peso: integer (nullable = true)
-- Tempo: integer (nullable = true)
-- TipoMotor: integer (nullable = true)
-- Transmissao: integer (nullable = true)
-- Marchas: integer (nullable = true)
-- Carburadors: integer (nullable = true)
-- HP: integer (nullable = true)

+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
Consumo|Cilindros|Cilindradas|RelEixoTraseiro|Peso|Tempo|TipoMotor|Transmissao|Marchas|Carburadors| HP|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
 21| 6| 160| 39| 262| 1646| 0| 1| 4| 4|110|
 21| 6| 160| 39|2875| 1702| 0| 1| 4| 4|110|
 228| 4| 108| 385| 232| 1861| 1| 1| 4| 1| 93|
 214| 6| 258| 308|3215| 1944| 1| 0| 3| 1|110|
 187| 8| 360| 315| 344| 1702| 0| 0| 3| 2|175|
 181| 6| 225| 276| 346| 2022| 1| 0| 3| 1|105|
 143| 8| 360| 321| 357| 1584| 0| 0| 3| 4|245|
 244| 4| 1467| 369| 319| 20| 1| 0| 4| 2| 62|
 228| 4| 1408| 392| 315| 229| 1| 0| 4| 2| 95|
 192| 6| 1676| 392| 344| 183| 1| 0| 4| 4|123|
 178| 6| 1676| 392| 344| 189| 1| 0| 4| 4|123|
 164| 8| 2758| 307| 407| 174| 0| 0| 3| 3|180|
 173| 8| 2758| 307| 373| 176| 0| 0| 3| 3|180|
 152| 8| 2758| 307| 378| 18| 0| 0| 3| 3|180|
 104| 8| 472| 293| 525| 1798| 0| 0| 3| 4|205|
 104| 8| 460| 3|5424| 1782| 0| 0| 3| 4|215|
 147| 8| 440| 323|5345| 1742| 0| 0| 3| 4|230|
 324| 4| 787| 408| 22| 1947| 1| 1| 4| 1| 66|
 304| 4| 757| 493|1615| 1852| 1| 1| 4| 2| 52|
 339| 4| 711| 422|1835| 199| 1| 1| 4| 1| 65|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
only showing top 20 rows

In [0]:
# Save your database to load it latter
# df1.write.format("csv").mode("overwrite").option("header", "true").option("inferSchema", True).save("/learn/Cars_Machine")

In [0]:
# Store the database previous loaded
cars_temp = spark.read.format("csv").option("header", True ).option("inferSchema", True).load("/learn/Cars_Machine") 
cars_temp.show()

+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
Consumo|Cilindros|Cilindradas|RelEixoTraseiro|Peso|Tempo|TipoMotor|Transmissao|Marchas|Carburadors| HP|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
 21| 6| 160| 39| 262| 1646| 0| 1| 4| 4|110|
 21| 6| 160| 39|2875| 1702| 0| 1| 4| 4|110|
 228| 4| 108| 385| 232| 1861| 1| 1| 4| 1| 93|
 214| 6| 258| 308|3215| 1944| 1| 0| 3| 1|110|
 187| 8| 360| 315| 344| 1702| 0| 0| 3| 2|175|
 181| 6| 225| 276| 346| 2022| 1| 0| 3| 1|105|
 143| 8| 360| 321| 357| 1584| 0| 0| 3| 4|245|
 244| 4| 1467| 369| 319| 20| 1| 0| 4| 2| 62|
 228| 4| 1408| 392| 315| 229| 1| 0| 4| 2| 95|
 192| 6| 1676| 392| 344| 183| 1| 0| 4| 4|123|
 178| 6| 1676| 392| 344| 189| 1| 0| 4| 4|123|
 164| 8| 2758| 307| 407| 174| 0| 0| 3| 3|180|
 173| 8| 2758| 307| 373| 176| 0| 0| 3| 3|180|
 152| 8| 2758| 307| 378| 18| 0| 0| 3| 3|180|
 104| 8| 472| 293| 525| 1798| 0| 0| 3| 4|205|
 104| 8| 460| 3|5424| 1782| 0| 0| 3| 4|215|
 147| 8| 440| 323|5345| 1742| 0| 0| 3| 4|230|
 324| 4| 787| 408| 22| 1947| 1| 1| 4| 1| 66|
 304| 4| 757| 493|1615| 1852| 1| 1| 4| 2| 52|
 339| 4| 711| 422|1835| 199| 1| 1| 4| 1| 65|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
only showing top 20 rows

In [0]:
cars_temp.show(5)

+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
Consumo|Cilindros|Cilindradas|RelEixoTraseiro|Peso|Tempo|TipoMotor|Transmissao|Marchas|Carburadors| HP|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
 21| 6| 160| 39| 262| 1646| 0| 1| 4| 4|110|
 21| 6| 160| 39|2875| 1702| 0| 1| 4| 4|110|
 228| 4| 108| 385| 232| 1861| 1| 1| 4| 1| 93|
 214| 6| 258| 308|3215| 1944| 1| 0| 3| 1|110|
 187| 8| 360| 315| 344| 1702| 0| 0| 3| 2|175|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
only showing top 5 rows

In [0]:
# Store it in a shorter DF
cars = cars_temp.select("Consumo", "Cilindros" , "Cilindradas" , "HP")
cars.show(10)

+-------+---------+-----------+---+
Consumo|Cilindros|Cilindradas| HP|
+-------+---------+-----------+---+
 21| 6| 160|110|
 21| 6| 160|110|
 228| 4| 108| 93|
 214| 6| 258|110|
 187| 8| 360|175|
 181| 6| 225|105|
 143| 8| 360|245|
 244| 4| 1467| 62|
 228| 4| 1408| 95|
 192| 6| 1676|123|
+-------+---------+-----------+---+
only showing top 10 rows

In [0]:
# Perform the One Hot Encode
vec_characteristics = VectorAssembler(inputCols=[("Consumo"),("Cilindros"), ("Cilindradas")] , outputCol= "characteristics")

In [0]:
# Transform our One Hot Encoded Data
cars =  vec_characteristics.transform(cars)

In [0]:
cars.show(7)

+-------+---------+-----------+---+-----------------+
Consumo|Cilindros|Cilindradas| HP| characteristics|
+-------+---------+-----------+---+-----------------+
 21| 6| 160|110| [21.0,6.0,160.0]|
 21| 6| 160|110| [21.0,6.0,160.0]|
 228| 4| 108| 93|[228.0,4.0,108.0]|
 214| 6| 258|110|[214.0,6.0,258.0]|
 187| 8| 360|175|[187.0,8.0,360.0]|
 181| 6| 225|105|[181.0,6.0,225.0]|
 143| 8| 360|245|[143.0,8.0,360.0]|
+-------+---------+-----------+---+-----------------+
only showing top 7 rows

In [0]:
# Defining the proportion of data to be tested and trained
cars_test , cars_train = cars.randomSplit([0.75 , 0.25])

In [0]:
# Verify the proportion
print(cars_test.count())
print()
print(cars_train.count())

25

7

In [0]:
# Doing a Liear Regression of HP  using characteristics as feature
lin_reg = LinearRegression(featuresCol="characteristics" , labelCol="HP")

In [0]:
# Fitting the model
model = lin_reg.fit(cars_train)

In [0]:
# Using our Linear Regression to Predict the HP values (prediction)
predict  = model.transform(cars_test)
predict.show(8)

+-------+---------+-----------+---+-----------------+------------------+
Consumo|Cilindros|Cilindradas| HP| characteristics| prediction|
+-------+---------+-----------+---+-----------------+------------------+
 15| 8| 301|335| [15.0,8.0,301.0]|157.77785977836666|
 21| 6| 160|110| [21.0,6.0,160.0]|109.69834049864602|
 26| 4| 1203| 91|[26.0,4.0,1203.0]| 41.3088902709217|
 104| 8| 460|215|[104.0,8.0,460.0]| 169.4321770088755|
 104| 8| 472|205|[104.0,8.0,472.0]|169.22796841034702|
 133| 8| 350|245|[133.0,8.0,350.0]| 175.9832167313713|
 143| 8| 360|245|[143.0,8.0,360.0]|177.42653516454638|
 147| 8| 440|230|[147.0,8.0,440.0]| 176.7105414138028|
+-------+---------+-----------+---+-----------------+------------------+
only showing top 8 rows

In [0]:
# We will Evaluate our model = > Comparing the HP (real value) with the predict one (preditions column)
evaluate = RegressionEvaluator(predictionCol="prediction" , labelCol="HP" , metricName="rmse" )

In [0]:
# Let´s see the result of the evaluation of our model
rmse = evaluate.evaluate(predict)
print(rmse)

53.44898821467219

In [0]:
# Apply the Random Forest Algorithm
forest = RandomForestRegressor(featuresCol="characteristics" , labelCol="HP")

In [0]:
# Creating the train model
model_forest = forest.fit(cars_train)

In [0]:
# Set the prection model
predict_forest = model_forest.transform(cars_test)
predict_forest.show()

+-------+---------+-----------+---+------------------+------------------+
Consumo|Cilindros|Cilindradas| HP| characteristics| prediction|
+-------+---------+-----------+---+------------------+------------------+
 15| 8| 301|335| [15.0,8.0,301.0]| 134.125|
 21| 6| 160|110| [21.0,6.0,160.0]| 115.275|
 26| 4| 1203| 91| [26.0,4.0,1203.0]| 105.325|
 104| 8| 460|215| [104.0,8.0,460.0]| 143.625|
 104| 8| 472|205| [104.0,8.0,472.0]| 143.625|
 133| 8| 350|245| [133.0,8.0,350.0]| 143.625|
 143| 8| 360|245| [143.0,8.0,360.0]| 143.625|
 147| 8| 440|230| [147.0,8.0,440.0]| 143.625|
 152| 8| 304|150| [152.0,8.0,304.0]| 143.625|
 152| 8| 2758|180|[152.0,8.0,2758.0]| 139.325|
 155| 8| 318|150| [155.0,8.0,318.0]| 143.625|
 158| 8| 351|264| [158.0,8.0,351.0]| 143.625|
 164| 8| 2758|180|[164.0,8.0,2758.0]| 139.325|
 173| 8| 2758|180|[173.0,8.0,2758.0]| 139.325|
 178| 6| 1676|123|[178.0,6.0,1676.0]| 130.625|
 192| 8| 400|175| [192.0,8.0,400.0]| 149.35|
 214| 4| 121|109| [214.0,4.0,121.0]|106.26666666666665|
 214| 6| 258|110| [214.0,6.0,258.0]|112.56666666666665|
 215| 4| 1201| 97|[215.0,4.0,1201.0]| 89.81666666666666|
 228| 4| 1408| 95|[228.0,4.0,1408.0]| 89.81666666666666|
+-------+---------+-----------+---+------------------+------------------+
only showing top 20 rows

In [0]:
# Evaluating our Random Forest Prediction
rmse_forest = evaluate.evaluate(predict_forest)
print(rmse_forest)

64.17122596356303